In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
from os.path import join
import torch
import sys
sys.path.append("/n/home12/binxuwang/Github/DiffusionObjectRelation/PixArt-alpha")
from diffusion import IDDPM
from diffusion.data.builder import build_dataset, build_dataloader, set_data_root
from diffusion.model.builder import build_model
from diffusion.utils.misc import set_random_seed, read_config, init_random_seed, DebugUnderflowOverflow
sys.path.append("/n/home12/binxuwang/Github/DiffusionObjectRelation/utils")
from pixart_utils import state_dict_convert
from image_utils import pil_images_to_grid
from diffusers import AutoencoderKL, Transformer2DModel, PixArtAlphaPipeline, DPMSolverMultistepScheduler

/n/home12/binxuwang/.conda/envs/torch2/lib/python3.10/site-packages/mmcv/__init__.py:20: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  warnings.warn(
/n/home12/binxuwang/.conda/envs/torch2/lib/python3.10/site-packages/transformers/utils/hub.py:127: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


In [3]:
from diffusers import AutoencoderKL, Transformer2DModel, PixArtAlphaPipeline, DPMSolverMultistepScheduler
@torch.inference_mode()
def visualize_prompts(pipeline, validation_prompts, prompt_cache_dir, max_length=120, weight_dtype=torch.float16,
                   num_inference_steps=14, guidance_scale=4.5, num_images_per_prompt=25, device="cuda"):
    # logger.info("Running validation... ")
    # device = accelerator.device
    # model = accelerator.unwrap_model(model)
    if validation_prompts is None:
        validation_prompts = [
            "triangle is to the upper left of square", 
            "blue triangle is to the upper left of red square", 
            "triangle is above and to the right of square", 
            "blue circle is above and to the right of blue square", 
            "triangle is to the left of square", 
            "triangle is to the left of triangle", 
            "circle is below red square",
            "red circle is to the left of blue square",
            "blue square is to the right of red circle",
            "red circle is above square",
            "triangle is above red circle",
            "red is above blue",
            "red is to the left of red",
            "blue triangle is above red triangle", 
            "blue circle is above blue square", 
        ]
    pipeline = pipeline.to(device)
    pipeline.set_progress_bar_config(disable=True)
    generator = torch.Generator(device=device).manual_seed(0)
    image_logs = []
    images = []
    latents = []
    uncond_data = torch.load(f'{prompt_cache_dir}/uncond_{max_length}token.pth', map_location='cpu')
    uncond_prompt_embeds = uncond_data['caption_embeds'].to(device)
    uncond_prompt_attention_mask = uncond_data['emb_mask'].to(device)
    visualized_prompts = []
    for _, prompt in enumerate(validation_prompts):
        if not os.path.exists(f'{prompt_cache_dir}/{prompt}_{max_length}token.pth'):
            continue
        embed = torch.load(f'{prompt_cache_dir}/{prompt}_{max_length}token.pth', map_location='cpu')
        caption_embs, emb_masks = embed['caption_embeds'].to(device), embed['emb_mask'].to(device)
        latents.append(pipeline(
            num_inference_steps=num_inference_steps,
            num_images_per_prompt=num_images_per_prompt,
            generator=generator,
            guidance_scale=guidance_scale,
            prompt_embeds=caption_embs,
            prompt_attention_mask=emb_masks,
            negative_prompt=None,
            negative_prompt_embeds=uncond_prompt_embeds,
            negative_prompt_attention_mask=uncond_prompt_attention_mask,
            use_resolution_binning=False, # need this for smaller images like ours. 
            output_type="latent",
        ).images)
        visualized_prompts.append(prompt)
    # flush()
    for latent in latents:
        images.append(pipeline.vae.decode(latent.to(weight_dtype) / pipeline.vae.config.scaling_factor, return_dict=False)[0])
    for prompt, image in zip(visualized_prompts, images):
        image = pipeline.image_processor.postprocess(image, output_type="pil")
        image_logs.append({"validation_prompt": prompt, "images": image})

    return image_logs

### Evaluation and extract mask of objects

### Hook embeddings

In [3]:
relations = [
    "above",
    "below",
    "to the left of",
    "to the right of",
    "to the upper left of",
    "to the upper right of",
    "to the lower left of",
    "to the lower right of",
]
# visualize prompts
new_prompt_cache_dir = "/n/holylfs06/LABS/kempner_fellow_binxuwang/Users/binxuwang/DL_Projects/PixArt/output/red_blue_8_position_rndembposemb"
visualize_prompts = [f"red is {relation} blue" for relation in relations] + [f"blue is {relation} red" for relation in relations]

In [4]:
visualize_prompts

['red is above blue',
 'red is below blue',
 'red is to the left of blue',
 'red is to the right of blue',
 'red is to the upper left of blue',
 'red is to the upper right of blue',
 'red is to the lower left of blue',
 'red is to the lower right of blue',
 'blue is above red',
 'blue is below red',
 'blue is to the left of red',
 'blue is to the right of red',
 'blue is to the upper left of red',
 'blue is to the upper right of red',
 'blue is to the lower left of red',
 'blue is to the lower right of red']

25

In [22]:
import pickle as pkl
from tqdm import tqdm, trange
from os.path import join
saveroot = "/n/holylfs06/LABS/kempner_fellow_binxuwang/Users/binxuwang/DL_Projects/PixArt/results/objrel_rndembdposemb_DiT_B_pilot/latent_store"
relations = [
    "above",
    "below",
    "to the left of",
    "to the right of",
    "to the upper left of",
    "to the upper right of",
    "to the lower left of",
    "to the lower right of",
]
visualize_prompts = [f"red is {relation} blue" for relation in relations] + [f"blue is {relation} red" for relation in relations]
# Initialize lists to store indices and vectors
# Create a dictionary to store all indices and vectors
data_dict = {
    'time_idx': [],
    'batch_idx': [], 
    'height_idx': [],
    'width_idx': [],
    'condpass_idx': [],
    'prompt_idx': [],
    'seed_idx': [],
    'repr_vectors': []
}

for random_seed in trange(10):
    for prompt_idx in range(len(visualize_prompts)):
        data = pkl.load(open(join(saveroot, f"red_blue_8_pos_rndembposemb_img_latent_residual_prompt{prompt_idx}_seed{random_seed}.pkl"), "rb"))
        image_logs = data["image_logs"]
        latents_traj = data["latents_traj"][0]
        pred_traj = data["pred_traj"][0]
        t_traj = data["t_traj"]
        residual_spatial_state_traj = data["block_11_residual_spatial_state_traj"]
        batch_size = len(image_logs[0]["images"])
        Tdim, B2dim, Hdim, Wdim, Ch = residual_spatial_state_traj.shape
        Tidx, B2idx, Hidx, Widx = torch.meshgrid(
            torch.arange(Tdim, dtype=torch.long),  # 14
            torch.arange(B2dim, dtype=torch.long),  # 50
            torch.arange(Hdim, dtype=torch.long),  # 8
            torch.arange(Wdim, dtype=torch.long),  # 8
            indexing='ij',  # "ij" means i0 corresponds to x.shape[0], i1 to x.shape[1], etc.
        ) # Each i0, i1, i2, i3 is now shape [14, 50, 8, 8].
        # Reshape indices
        Tidx = Tidx.reshape(-1)
        B2idx = B2idx.reshape(-1)
        condpass_idx = (B2idx >= batch_size)
        Hidx = Hidx.reshape(-1)
        Widx = Widx.reshape(-1)
        # Add to data dictionary
        data_dict['time_idx'].append(Tidx)
        data_dict['batch_idx'].append(B2idx)
        data_dict['height_idx'].append(Hidx)
        data_dict['width_idx'].append(Widx)
        data_dict['condpass_idx'].append(condpass_idx)
        data_dict['prompt_idx'].append(prompt_idx * torch.ones_like(Tidx, dtype=torch.long))
        data_dict['seed_idx'].append(random_seed * torch.ones_like(Tidx, dtype=torch.long))
        data_dict['repr_vectors'].append(residual_spatial_state_traj.reshape(Tdim*B2dim*Hdim*Wdim, Ch))
        
        # print(residual_spatial_state_traj.shape) # ([14, 50, 8, 8, 768])
        # print(pred_traj.shape) # ([14, 25, 4, 16, 16])
        # print(latents_traj.shape) # ([15, 25, 4, 16, 16])
        # print(len(t_traj[0]))
    #     break
    # break

# Concatenate all data
final_data = {
    key: torch.cat(values, dim=0) 
    for key, values in data_dict.items()
}

  0%|          | 0/10 [00:00<?, ?it/s]

100%|██████████| 10/10 [00:34<00:00,  3.46s/it]


In [24]:
pkl.dump(final_data, open(join(saveroot, "red_blue_8_pos_rndembposemb_repr_vector_dataset_all.pkl"), "wb"))

In [25]:
!du -sh /n/holylfs06/LABS/kempner_fellow_binxuwang/Users/binxuwang/DL_Projects/PixArt/results/objrel_rndembdposemb_DiT_B_pilot/latent_store/red_blue_8_pos_rndembposemb_repr_vector_dataset_all.pkl

21G	/n/holylfs06/LABS/kempner_fellow_binxuwang/Users/binxuwang/DL_Projects/PixArt/results/objrel_rndembdposemb_DiT_B_pilot/latent_store/red_blue_8_pos_rndembposemb_repr_vector_dataset_all.pkl


In [23]:
for k, v in final_data.items():
    print(k, v.shape)

time_idx torch.Size([7168000])
batch_idx torch.Size([7168000])
height_idx torch.Size([7168000])
width_idx torch.Size([7168000])
condpass_idx torch.Size([7168000])
prompt_idx torch.Size([7168000])
seed_idx torch.Size([7168000])
repr_vectors torch.Size([7168000, 768])


In [14]:
import math
math.prod([14, 50, 8, 8])

44800

In [11]:
len(t_traj[0])

14